In [1]:
!pip install sqlalchemy  pandas


[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from psycopg2 import connect
from dotenv import load_dotenv
from sqlalchemy import create_engine

import pandas as pd
import os

In [2]:
load_dotenv()
dbname = os.getenv('DBNAME')
user = os.getenv('USER')
password = os.getenv('PASSWORD')
host = os.getenv('HOST')
port = os.getenv('PORT')

In [3]:
def db_connect():
    """Establishes a connection to the database."""
    return connect(
        dbname = dbname,
        user = user,
        password = password,
        host = host,
        port = port
    )

In [4]:
conn = db_connect()

In [5]:
engine = create_engine(f'postgresql://{user}:{password}@{host}:{port}/{dbname}')

In [6]:
#query = """
#    SELECT 
#        department_id,
#        job_id,
#        EXTRACT(QUARTER FROM TO_DATE(datetime, 'YYYY-MM-DD')) AS quarter,
#        COUNT(id) AS num_hired
#    FROM public.employees
#    WHERE EXTRACT(YEAR FROM TO_DATE(datetime, 'YYYY-MM-DD')) = 2021
#    GROUP BY department_id, job_id, quarter
#    ORDER BY department_id ASC, job_id ASC;
#"""

In [ ]:
query_1 = """
    SELECT 
        d.department, 
        j.job,
        COUNT(e.id) FILTER (WHERE EXTRACT(QUARTER FROM TO_DATE(e.datetime, 'YYYY-MM-DD')) = 1) AS Q1,
        COUNT(e.id) FILTER (WHERE EXTRACT(QUARTER FROM TO_DATE(e.datetime, 'YYYY-MM-DD')) = 2) AS Q2,
        COUNT(e.id) FILTER (WHERE EXTRACT(QUARTER FROM TO_DATE(e.datetime, 'YYYY-MM-DD')) = 3) AS Q3,
        COUNT(e.id) FILTER (WHERE EXTRACT(QUARTER FROM TO_DATE(e.datetime, 'YYYY-MM-DD')) = 4) AS Q4
    FROM public.employees e
    JOIN public.jobs j ON e.job_id = j.id
    JOIN public.departments d ON e.department_id = d.id
    WHERE EXTRACT(YEAR FROM TO_DATE(e.datetime, 'YYYY-MM-DD')) = 2021
    GROUP BY d.department, j.job
    ORDER BY d.department ASC, j.job ASC;
"""

In [ ]:
df_1 = pd.read_sql(query_1, engine)

In [ ]:
df_1

,department,job,q1,q2,q3,q4
0,Accounting,Account Representative IV,1,0,0,0
1,Accounting,Actuary,0,1,0,0
2,Accounting,Analyst Programmer,0,0,1,0
3,Accounting,Budget/Accounting Analyst III,0,1,0,0
4,Accounting,Cost Accountant,0,1,0,0
...,...,...,...,...,...,...
928,Training,Teacher,0,2,0,0
929,Training,Technical Writer,0,0,1,0
930,Training,VP Product Management,1,0,0,0
931,Training,VP Quality Control,0,1,0,0


In [ ]:
df_1.to_csv('employees_department.csv', index=False)

In [34]:
query_2 = """
    SELECT 
        d.id AS department_id,
        d.department AS department_name,
        COUNT(e.id) AS employees_hired
    FROM departments d
    JOIN employees e ON d.id = e.department_id
    WHERE EXTRACT(YEAR FROM TO_DATE(e.datetime, 'YYYY-MM-DD')) = 2021
    GROUP BY d.id, d.department
    HAVING COUNT(e.id) > (
        SELECT AVG(employee_count) 
        FROM (
            SELECT COUNT(id) AS employee_count
            FROM employees  -- Se eliminó el alias "e"
            WHERE EXTRACT(YEAR FROM TO_DATE(datetime, 'YYYY-MM-DD')) = 2021
            GROUP BY department_id
        ) AS dept_hiring_avg
    )
    ORDER BY employees_hired DESC;
"""

In [35]:
df_2 = pd.read_sql(query_2, engine)

In [36]:
df_2

,department_id,department_name,employees_hired
0,8,Support,220
1,5,Engineering,206
2,7,Services,202
3,6,Human Resources,201
4,4,Business Development,185
5,3,Research and Development,150
6,9,Marketing,143


In [37]:
df_2.to_csv('hiring_department.csv')